In [ ]:
!pip install kaggle


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from google.colab import files
files.upload()


Saving fracture_datset.zip to fracture_datset.zip


In [ ]:
!mkdir -p ~/.kaggle
!mv kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json


In [ ]:
!kaggle datasets download -d vuppalaadithyasairam/bone-fracture-detection-using-xrays!

Traceback (most recent call last):
  File "/usr/local/bin/kaggle", line 5, in <module>
    from kaggle.cli import main
  File "/usr/local/lib/python3.10/dist-packages/kaggle/__init__.py", line 23, in <module>
    api.authenticate()
  File "/usr/local/lib/python3.10/dist-packages/kaggle/api/kaggle_api_extended.py", line 169, in authenticate
    self._load_config(config_data)
  File "/usr/local/lib/python3.10/dist-packages/kaggle/api/kaggle_api_extended.py", line 210, in _load_config
    raise ValueError('Error: Missing %s in configuration.' % item)
ValueError: Error: Missing username in configuration.


In [ ]:
!unzip /content/drive/MyDrive/fracture_datset.zip


Streaming output truncated to the last 5000 lines.
  inflating: archive (6)/train/fractured/99-rotated2-rotated3-rotated3.jpg  
  inflating: archive (6)/train/fractured/99-rotated2-rotated3.jpg  
  inflating: archive (6)/train/fractured/99-rotated2.jpg  
  inflating: archive (6)/train/fractured/99-rotated3-rotated1-rotated1.jpg  
  inflating: archive (6)/train/fractured/99-rotated3-rotated1-rotated2.jpg  
  inflating: archive (6)/train/fractured/99-rotated3-rotated1-rotated3.jpg  
  inflating: archive (6)/train/fractured/99-rotated3-rotated1.jpg  
  inflating: archive (6)/train/fractured/99-rotated3-rotated2-rotated1.jpg  
  inflating: archive (6)/train/fractured/99-rotated3-rotated2-rotated2.jpg  
  inflating: archive (6)/train/fractured/99-rotated3-rotated2-rotated3.jpg  
  inflating: archive (6)/train/fractured/99-rotated3-rotated2.jpg  
  inflating: archive (6)/train/fractured/99-rotated3-rotated3-rotated1.jpg  
  inflating: archive (6)/train/fractured/99-rotated3-rotated3-rotated2

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/content'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All"
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

Streaming output truncated to the last 5000 lines.
/content/archive (6)/train/fractured/70-rotated1-rotated2-rotated2.jpg
/content/archive (6)/train/fractured/41-rotated1-rotated1-rotated3.jpg
/content/archive (6)/train/fractured/60-rotated3-rotated2-rotated2.jpg
/content/archive (6)/train/fractured/69-rotated1-rotated2.jpg
/content/archive (6)/train/fractured/92-rotated3-rotated2-rotated1.jpg
/content/archive (6)/train/fractured/80-rotated3-rotated1-rotated1.jpg
/content/archive (6)/train/fractured/111-rotated1-rotated2-rotated1.jpg
/content/archive (6)/train/fractured/120-rotated3-rotated3.jpg
/content/archive (6)/train/fractured/23-rotated1-rotated3-rotated1.jpg
/content/archive (6)/train/fractured/97-rotated2-rotated3.jpg
/content/archive (6)/train/fractured/95-rotated3-rotated1-rotated2.jpg
/content/archive (6)/train/fractured/25-rotated1-rotated1-rotated3.jpg
/content/archive (6)/train/fractured/65-rotated3-rotated1.jpg
/content/archive (6)/train/fractured/29-rotated2-rotated1-ro

In [ ]:
import numpy as np
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
from keras import Sequential
from tensorflow.keras.layers import *
from tensorflow.keras.models import *
from tensorflow.keras.preprocessing import image

In [ ]:
train_path = '/content/archive (6)/train'
test_path = '/content/archive (6)/val'

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
train_datagen = image.ImageDataGenerator(
    rotation_range=15,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest',
    width_shift_range=0.1,
    height_shift_range=0.1
)
val_datagen= image.ImageDataGenerator(
    rotation_range=15,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest',
    width_shift_range=0.1,
    height_shift_range=0.1)

In [ ]:
train_generator = train_datagen.flow_from_directory(
    train_path,
    target_size = (224,224),
    batch_size = 4,
    class_mode = 'binary')
validation_generator = val_datagen.flow_from_directory(
    test_path,
    target_size = (224,224),
    batch_size = 4,
    shuffle=True,
    class_mode = 'binary')

Found 8863 images belonging to 2 classes.
Found 600 images belonging to 2 classes.


In [ ]:
base_model = tf.keras.applications.EfficientNetB3(weights='imagenet', input_shape=(224,224,3), include_top=False)

for layer in base_model.layers:
    layer.trainable=False
model = Sequential()
model.add(base_model)
model.add(GaussianNoise(0.25))
model.add(GlobalAveragePooling2D())
model.add(Dense(512,activation='relu'))
model.add(BatchNormalization())
model.add(GaussianNoise(0.25))
model.add(Dropout(0.25))
model.add(Dense(1, activation='sigmoid'))
model.summary()

43941136/43941136 [==============================] - 0s 0us/step
Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 efficientnetb3 (Functional)  (None, 7, 7, 1536)       10783535  
                                                                 
 gaussian_noise (GaussianNoi  (None, 7, 7, 1536)       0         
 se)                                                             
                                                                 
 global_average_pooling2d (G  (None, 1536)             0         
 lobalAveragePooling2D)                                          
                                                                 
 dense (Dense)               (None, 512)               786944    
                                                                 
 batch_normalization (BatchN  (None, 512)              2048      
 ormalization)                                           

In [ ]:
model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy','Precision','Recall','AUC'])

from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
lrp=ReduceLROnPlateau(monitor="val_loss", factor=0.1, patience=2)
filepath='best_model.h5'
checkpoint = ModelCheckpoint(filepath, monitor='val_accuracy', verbose=1, save_best_only=True, mode='max')
call=[checkpoint,lrp]
history = model.fit(
    train_generator,
    epochs=10,
    validation_data=validation_generator,
    steps_per_epoch= 50,
    callbacks=call
    )

from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau

lrp = ReduceLROnPlateau(monitor="val_loss", factor=0.1, patience=2)
filepath = 'best_model.h5'
checkpoint = ModelCheckpoint(filepath, monitor='val_accuracy', verbose=1, save_best_only=True, mode='max')
call = [checkpoint, lrp]

# Modify the monitor value of ReduceLROnPlateau callback
lrp_monitor = 'val_loss'  # Use a different metric that doesn't involve tensors

# Train the model
history = model.fit(
    train_generator,
    epochs=10,
    validation_data=validation_generator,
    steps_per_epoch=50,
    callbacks=call
)

In [ ]:
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau

lrp = ReduceLROnPlateau(monitor="val_loss", factor=0.1, patience=2)
filepath = 'best_model.h5'

# Modify the monitor value of ModelCheckpoint callback
checkpoint_monitor = 'val_loss'  # Use a different metric that doesn't involve tensors

checkpoint = ModelCheckpoint(filepath, monitor=checkpoint_monitor, verbose=1, save_best_only=True,mode='min',save_weights_only=True )
call = [checkpoint, lrp]

# Train the model
history = model.fit(
    train_generator,
    epochs=10,
    validation_data=validation_generator,
    steps_per_epoch=50,
    callbacks=call
)


Epoch 1/10
50/50 [==============================] - ETA: 0s - loss: 1.0997 - accuracy: 0.6100 - precision: 0.6022 - recall: 0.5773 - auc: 0.6372
Epoch 1: val_loss improved from inf to 0.70992, saving model to best_model.h5
50/50 [==============================] - 198s 4s/step - loss: 1.0997 - accuracy: 0.6100 - precision: 0.6022 - recall: 0.5773 - auc: 0.6372 - val_loss: 0.7099 - val_accuracy: 0.5267 - val_precision: 0.4091 - val_recall: 0.4125 - val_auc: 0.5704 - lr: 0.0010
Epoch 2/10
50/50 [==============================] - ETA: 0s - loss: 0.7203 - accuracy: 0.6350 - precision: 0.6250 - recall: 0.6186 - auc: 0.6970
Epoch 2: val_loss did not improve from 0.70992
50/50 [==============================] - 175s 4s/step - loss: 0.7203 - accuracy: 0.6350 - precision: 0.6250 - recall: 0.6186 - auc: 0.6970 - val_loss: 0.7857 - val_accuracy: 0.5667 - val_precision: 0.4664 - val_recall: 0.5792 - val_auc: 0.6137 - lr: 0.0010
Epoch 3/10
50/50 [==============================] - ETA: 0s - loss: 0.6

In [ ]:
model.evaluate(train_generator)

2216/2216 [==============================] - 1417s 640ms/step - loss: 0.4334 - accuracy: 0.8000 - precision: 0.7847 - recall: 0.8207 - auc: 0.8818


[0.43342310190200806,
 0.799954891204834,
 0.7846858501434326,
 0.8206707835197449,
 0.8817657828330994]

In [ ]:
model.evaluate(validation_generator)
model.load_weights('my_model.keras')

150/150 [==============================] - 107s 716ms/step - loss: 0.7283 - accuracy: 0.6600 - precision: 0.5726 - recall: 0.5917 - auc: 0.7242


In [ ]:
model.save('Best_model')

TypeError: ignored

In [ ]:
from tensorflow.keras.utils import load_img, img_to_array
import numpy as np
img = load_img('/content/archive (6)/train/not fractured/10-rotated1-rotated1.jpg',target_size=(224,224))
imag = img_to_array(img)
imaga = np.expand_dims(imag,axis=0)
new_model = tf.keras.models.load_('/content/best_model.h5')
ypred = new_model.predict(imaga)
print(ypred)
a=ypred[0]
if a<0.5:
      op="Fracture"
else:
      op="Normal"
plt.imshow(img)
print("THE UPLOADED X-RAY IMAGE IS: "+str(op))

AttributeError: ignored